In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../Models')
from utils import PrimalDualSolver_l2, ProjectedGradientDescent_l2


class SDL_simple:
    """
    Supervised Dictionary Learning (SDL) Class.

    Combines sparse coding with supervised learning by jointly learning:
    - A dictionary `D` for sparse representation.
    - A linear model `(theta, b)` for predicting labels.
    """

    def __init__(self, n_iter=10,
                 lamnda0=0.01,
                 lambda1=0.1,
                 lambda2=0.1,
                 lr_D=0.01,
                 lr_theta=0.01,
                 lr_alpha=0.01,
                 lambd=0.01):
        self.n_iter = n_iter
        self.lamnda0 = lamnda0
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lr_D = lr_D
        self.lr_theta = lr_theta
        self.lr_alpha = lr_alpha
        self.lambd = lambd

    def objective(self, X, y, D, theta, b, alpha):
        """Computes the objective function value with separate terms."""
        total_loss_dict = 0
        total_loss_class = 0
        total_sparse_penalty = 0

        for i in range(X.shape[0]):
            xi, yi, ai = X[i], y[i], alpha[i]

            # Compute each term
            loss_dict = np.linalg.norm(xi - D @ ai)**2
            loss_class = np.linalg.norm(yi - (theta @ ai + b))**2
            sparse_penalty = self.lambda1 * np.linalg.norm(ai, 1)

            # Accumulate terms
            total_loss_dict += loss_dict
            total_loss_class += loss_class
            total_sparse_penalty += sparse_penalty

        # Combine the terms
        total_objective = total_loss_dict + total_loss_class + total_sparse_penalty
        return total_objective

    def loss_prediction(self, X, y, D, theta, b, alpha):
        """Computes the classification loss."""
        total_loss = 0
        for i in range(X.shape[0]):
            xi, yi, ai = X[i], y[i], alpha[i]
            loss = np.linalg.norm(yi - (theta @ ai + b))**2
            total_loss += loss
        return total_loss

    def solve_alpha(self, X, y, D, theta, b):
        """Optimizes sparse codes `alpha` for fixed `D` and `theta`."""
        n_samples, n_features = X.shape
        alpha = np.zeros((n_samples, n_features))

        for i in range(n_samples):
            x_i = X[i]
            y_i = y[i]

            solver = PrimalDualSolver_l2(
                theta=theta, b=b, x_i=x_i, y_i=y_i, D=D,
                lambda_0=self.lamnda0, lambda_1=self.lambda1,
                lambd=self.lambd, mu=1.0
            )
            # Solve the problem
            x0 = np.random.randn(n_features)  # Random initialization
            alpha_opt, _ = solver.solve(x0)

            alpha[i] = alpha_opt

        return alpha

    def solve_D_theta(self, alpha_opt, X, y, D_opt, theta_opt, b):
        """Updates `D` and `theta` given the optimal `alpha`."""
        pgd = ProjectedGradientDescent_l2(
            D_init=D_opt, theta_init=theta_opt,
            b=b, x=X, y=y, alphas=alpha_opt,
            lambda_0=self.lamnda0,
            lambda_1=self.lambda1, lambda_2=self.lambda2,
            lr=self.lr_D, max_iter=self.n_iter
        )
        D_opt, theta_opt, b_opt, _ = pgd.optimize()
        return D_opt, theta_opt, b_opt

    def fit(self, X, y):
        """Fits the model to the data."""
        n_samples, n_features = X.shape
        self.n_components = n_features
        D_opt = np.random.randn(n_features, self.n_components)
        D_opt /= np.linalg.norm(D_opt, axis=0)
        theta_opt = np.zeros(self.n_components)
        b_opt = 0

        for i in range(self.n_iter):
            alpha_opt = self.solve_alpha(X, y, D_opt, theta_opt, b_opt)
            D_opt, theta_opt, b_opt = self.solve_D_theta(alpha_opt,
                                                            X,
                                                            y,
                                                            D_opt,
                                                            theta_opt,
                                                            b_opt)
            # Compute the loss
            loss = self.objective(X, y, D_opt, theta_opt, b_opt, alpha_opt)
            print(f"Iteration {i+1}/{self.n_iter}, Loss: {loss}")
            print(f"Iteration {i+1}/{self.n_iter}, Loss classification: {self.loss_prediction(X, y, D_opt, theta_opt, b_opt, alpha_opt)}")

            # Stop the model if the loss is NaN
            if np.isnan(loss):
                print("Loss is NaN. Stopping optimization.")
                break

        self.alpha = alpha_opt
        self.D = D_opt
        self.theta = theta_opt
        self.b = b_opt

    def predict(self, X):
        """Predicts labels for input data `X`."""
        predictions = []
        for i in range(X.shape[0]):
            x_i = X[i]
            alpha, _, _, _ = np.linalg.lstsq(self.D, x_i, rcond=None)
            prediction = self.theta @ alpha + self.b
            predictions.append(prediction)
        return np.sign(np.array(predictions))

    def score(self, X, y):
        """Computes classification accuracy."""
        y_pred = self.predict(X)
        return np.mean(np.round(y_pred) == y)


In [16]:

# Generate synthetic data
np.random.seed(42)
n_samples, n_features, n_components = 100, 40, 10
X = np.random.randn(n_samples, n_features)  # Input data
true_D = np.random.randn(n_features, n_components)  # True dictionary
true_alpha = np.random.randn(n_samples, n_components)  # Sparse codes
true_theta = np.random.randn(n_components)  # Linear model weights
b_true = 0.5  # Bias term


# Generate labels based on the linear model
y = X @ true_D @ true_theta + b_true + 0.1 * np.random.randn(n_samples)
y = np.round(y)  # Binary classification labels (0 or 1)


# Initialize the SDL model
sdl = SDL_simple(
    n_iter=10,
    lamnda0=0.01,
    lambda1=0.1,
    lambda2=0.1,
    lr_D=0.01,
    lr_theta=0.01,
    lr_alpha=0.01,
    lambd=0.01
)

# Fit the model to the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sdl.fit(X_train, y_train)


Iteration 1/10, Loss: 25204.422782856516
Iteration 1/10, Loss classification: 22103.430123526443
Iteration 2/10, Loss: 3116.3615197649797
Iteration 2/10, Loss classification: 0.36066456177186995
Iteration 3/10, Loss: 3099.8651714636976
Iteration 3/10, Loss classification: 0.3834475388819589
Iteration 4/10, Loss: 3071.527807386007
Iteration 4/10, Loss classification: 0.3402792126546438
Iteration 5/10, Loss: 3064.8620173396885
Iteration 5/10, Loss classification: 0.33225744263924245
Iteration 6/10, Loss: 3071.5777221677904
Iteration 6/10, Loss classification: 0.33315375791015167
Iteration 7/10, Loss: 3057.732594020944
Iteration 7/10, Loss classification: 0.308996416352352
Iteration 8/10, Loss: 3048.6964875896083
Iteration 8/10, Loss classification: 0.32826066956479305
Iteration 9/10, Loss: 3045.175036798881
Iteration 9/10, Loss classification: 0.311010803775823
Iteration 10/10, Loss: 3057.8753467177103
Iteration 10/10, Loss classification: 0.30132643715145957


In [17]:

# do the prediction
y_pred_train = sdl.predict(X_train)
print("Train accuracy:", np.mean(np.round(y_pred_train) == y_train))

y_pred_test = sdl.predict(X_test)
print("Test accuracy:", np.mean(np.round(y_pred_test) == y_test))
print("Ground truth labels:", y_test[:10])
print("Predicted labels:", np.round(y_pred_test)[:10])

# count the number of right predictions
print("Number of right predictions:", np.sum(np.round(y_pred_test) == y_test))


Train accuracy: 1.0
Test accuracy: 0.0
Ground truth labels: [ 35.  -5.  16. -36.  -0. -25.  25.   2.   2.   7.]
Predicted labels: [ 10. -13. -21. -24. -25. -14. -25.  -3.  29.  -8.]
Number of right predictions: 0


In [9]:
from itertools import product

def select_best_lambda(X, y, lambda0_range, lambda2_range, n_iter=10):
    """
    Selects the best lambda parameters for the SDL model using grid search with a fixed ratio between lambda1 and lambda0.

    Parameters:
    - X, y: Dataset for training and evaluation.
    - lambda0_range: Range of values to test for lambda0.
    - lambda2_range: Range of values to test for lambda2.
    - n_iter: Number of iterations for the SDL model.

    Returns:
    - best_params: Dictionary with the best lambda parameters.
    - best_accuracy: Highest accuracy achieved.
    """
    best_accuracy = -float('inf')
    best_params = {}

    # Total number of combinations to evaluate
    total_combinations = len(lambda0_range) * len(lambda2_range)
    combination_count = 0

    for lambda0 in lambda0_range:
        # Calculate lambda1 such that lambda1/lambda0 is approximately 0.15
        lambda1 = 0.15 * lambda0

        for lambda2 in lambda2_range:
            # Increment the combination counter
            combination_count += 1

            # Initialize and fit the SDL model
            sdl = SDL_simple(
                n_iter=n_iter,
                lamnda0=lambda0,
                lambda1=lambda1,
                lambda2=lambda2,
                lr_D=0.01,
                lr_theta=0.01,
                lr_alpha=0.01
            )
            sdl.fit(X, y)

            # Evaluate the model
            accuracy = sdl.score(X, y)
            print(f"Progress: {combination_count}/{total_combinations} ({(combination_count/total_combinations)*100:.2f}%)")
            print(f"Lambda0: {lambda0}, Lambda1: {lambda1}, Lambda2: {lambda2}, Accuracy: {accuracy:.2f}")

            # Update best parameters if the current accuracy is better
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'lambda0': lambda0, 'lambda1': lambda1, 'lambda2': lambda2}

    print(f"Best Parameters: {best_params}, Best Accuracy: {best_accuracy:.2f}")
    return best_params, best_accuracy

# Example usage
lambda0_range = [0.001, 0.01, 0.1]
lambda2_range = [0.01, 0.1, 0.5]

best_params, best_accuracy = select_best_lambda(X, y, lambda0_range, lambda2_range)


Iteration 1/10, Loss: 199507201456356.2


/Users/christophermarouani/Desktop/SDL/Supervised_Dictionary_Learning/Models/utils.py:80: RuntimeWarning: invalid value encountered in subtract
  alpha_new = self.mu * self.prox_l1(alpha - self.lambd * grad_f, self.lambd * self.lambda_1)


Iteration 2/10, Loss: nan
Loss is NaN. Stopping optimization.
Progress: 1/9 (11.11%)
Lambda0: 0.001, Lambda1: 0.00015, Lambda2: 0.01, Accuracy: 0.00


KeyboardInterrupt: 